# Voting Mechanism Simulation Examples

Our goal here is to show basic use cases for the code, and do basic proof-of-concept on possible future extensions.  

In [20]:
# Standard Imports
import numpy as np
import pandas as pd
import os
from random import choice
from typing import Dict

import sys
sys.path.append('..')  # Add this line to include the directory above

# Custom imports
from custom_types import UserNFTs

from mechanisms.single_choice_weighted_plurality import SingleChoiceWeightedPlurality
from mechanisms.group_hug_mechanism import GroupHug

In [18]:
file_name = "../data/nft_data_may_282024__cleaned.csv"
file_exists = os.path.exists(file_name)
file_exists

False

# Key Question

How often would a vote that incorporates credibility-weighting by NFTs produce a different outcome than simply doing one wallet, one vote? We use the actual TE Academy NFT data as of May 28 to look at this question. 

## Inputting Sample Data as a Dictionary

In [ ]:
VOTER_DATA_FILENAME = "../data/nft_data_may_28_2024_cleaned.csv"

voter_data = pd.read_csv(VOTER_DATA_FILENAME)

# We may need to drop a specific column. 
voter_data.drop(columns = ['Unnamed: 0'], 
                inplace = True)
voter_data.set_index('ID', inplace = True)
sample_voters = voter_data.to_dict(orient='index')

In [ ]:
sample_voters

In [ ]:
voters = {key: UserNFTs(sample_voters.get(key))
          for key, _ in sample_voters.items()
          }

In [ ]:
voters

In [ ]:
# Get the first voter
first_voter = list(sample_voters.values())[0]

NFT_weights = {key: 1.0 
               for key
               in first_voter.keys()
               }

In [ ]:
def calc_voter_weights_from_NFT_weight(voters: Dict,
                                       nft_weights: Dict) -> Dict:
    new_dict = {}
    for voter in voters.keys():
        new_dict[voter] = {}
        new_dict[voter]["weight"] = 0
        for nft_name, nft_val in nft_weights.items():
            if voters.get(voter).get(nft_name):
                new_dict[voter]["weight"] += nft_val

    return new_dict

In [ ]:
voter_weights = calc_voter_weights_from_NFT_weight(voters,
                                                   NFT_weights)

In [ ]:
voter_weights

In [ ]:
uniform_weights = {key: {"weight": 1.0} for key in sample_voters.keys()}

In [ ]:
uniform_weights

In [14]:
SCWPCalculator = SingleChoiceWeightedPlurality()

NUM_EXPERIMENTS = 100_000

# Create an empty DataFrame to store simulation results
results_list = [] 

for k in range(NUM_EXPERIMENTS):
    sample_choices = {
                   key: choice(["A","B","C","D"])
                   for key in sample_voters.keys()
                 }
    weighted_winner, weighted_results  = SCWPCalculator.calculate(voter_weights,
                            sample_choices)
    uniform_winner, uniform_results = SCWPCalculator.calculate(uniform_weights,
                            sample_choices)
    # Create an empty DataFrame to store simulation results
    results_list.append({'Experiment': k,
                        'weighted_winner': weighted_winner, 
                        'weighted_candidate_scores': weighted_results,
                        'uniform_winner': uniform_winner,
                        'uniform_candidate_scores': uniform_results}
                        )


results_df = pd.DataFrame(results_list)

#TODO: Refactor for speed if needed.  

    

In [15]:
results_df

,Experiment,weighted_winner,weighted_candidate_scores,uniform_winner,uniform_candidate_scores
0,0,D,"{'D': 230.0, 'B': 211.0, 'C': 217.0, 'A': 203.0}",A,"{'D': 86.0, 'B': 88.0, 'C': 84.0, 'A': 89.0}"
1,1,A,"{'A': 271.0, 'D': 155.0, 'C': 241.0, 'B': 194.0}",A,"{'A': 99.0, 'D': 77.0, 'C': 95.0, 'B': 76.0}"
2,2,B,"{'B': 240.0, 'D': 240.0, 'A': 225.0, 'C': 156.0}",B,"{'B': 99.0, 'D': 94.0, 'A': 87.0, 'C': 67.0}"
3,3,D,"{'A': 157.0, 'B': 210.0, 'D': 266.0, 'C': 228.0}",D,"{'A': 68.0, 'B': 86.0, 'D': 106.0, 'C': 87.0}"
4,4,A,"{'A': 260.0, 'C': 204.0, 'D': 179.0, 'B': 218.0}",A,"{'A': 114.0, 'C': 80.0, 'D': 71.0, 'B': 82.0}"
...,...,...,...,...,...
99995,99995,A,"{'B': 209.0, 'C': 152.0, 'D': 233.0, 'A': 267.0}",A,"{'B': 85.0, 'C': 75.0, 'D': 91.0, 'A': 96.0}"
99996,99996,B,"{'B': 248.0, 'D': 152.0, 'A': 231.0, 'C': 230.0}",B,"{'B': 95.0, 'D': 66.0, 'A': 95.0, 'C': 91.0}"
99997,99997,A,"{'D': 240.0, 'B': 199.0, 'A': 267.0, 'C': 155.0}",A,"{'D': 85.0, 'B': 79.0, 'A': 102.0, 'C': 81.0}"
99998,99998,A,"{'B': 225.0, 'A': 263.0, 'D': 170.0, 'C': 203.0}",A,"{'B': 83.0, 'A': 107.0, 'D': 71.0, 'C': 86.0}"


In [16]:
(results_df['weighted_winner'] == results_df['uniform_winner']).mean()

0.67135

In [17]:
import mechanisms.group_hug_mechanism as GH